In [ ]:
import sys
sys.path.append('/home/shenqi/Master_thesis/SEED')
from Model.feature_extractor import SEED_EventGRU,SEED_EventGRUFuseDownsampleConv_DoubleConditionalConv
from Model.ssd_head import BoxHead
from metavision_ml.detection.anchors import Anchors
from utils.dataloader import seq_dataloader
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from metavision_ml.detection.anchors import Anchors
from Model.detection import inference_step
from Model.detection import evaluate
from utils.dataloader import seq_dataloader
# from torch_receptive_field import receptive_field, receptive_field_for_unit
import matplotlib.pyplot as plt


dataset_path = '/media/shenqi/data/Gen4_multi_timesurface_FromDat_small'
dataset_type = 'gen4'
dataloader = seq_dataloader(dataset_path = dataset_path, dataset_type = dataset_type, num_tbins = 1, batch_size = 1, channels = 6)

cout = 128
threshold_group = 1
net = SEED_EventGRUFuseDownsampleConv_DoubleConditionalConv(dataloader.channels, base=int(cout/16), cout=cout, dataset = dataset_type, pruning = False,threshold_group=threshold_group)
box_coder = Anchors(num_levels=net.levels, anchor_list='PSEE_ANCHORS', variances=[0.1, 0.2])
ssd_head = BoxHead(net.cout, box_coder.num_anchors, len(dataloader.wanted_keys)+1, n_layers=0)
net.eval().to('cuda')
ssd_head.eval().to('cuda')
net.load_state_dict(torch.load('/home/shenqi/Master_thesis/SEED/Saved_Model/gen4/SEED_EventGRUFuseDownsampleConv_DoubleConditionalConv/35_model.pth',map_location=torch.device('cuda')))
ssd_head.load_state_dict(torch.load('/home/shenqi/Master_thesis/SEED/Saved_Model/gen4/SEED_EventGRUFuseDownsampleConv_DoubleConditionalConv/35_pd.pth',map_location=torch.device('cuda')))



In [ ]:
output_val_list = []
cnt_val = 0
mean_activity_egru_ave = [0] * net.levels
mean_activity_conv1_ave = [0] * 7
mean_activity_egruRelu_ave = [0] * net.levels
box_hid_mean_ave = [0] * net.levels
cls_hid_mean_ave = [0] * net.levels
receptive_field_input = torch.zeros((7,360,640))

with tqdm(total=len(dataloader.seq_dataloader_test), desc=f'Validation',ncols=120) as pbar:
            
    for data in dataloader.seq_dataloader_test:
        sys.stdout.flush()
        with torch.no_grad():
            cnt_val += 1
            if cnt_val>20:
                raise Exception
            data['inputs'] = data['inputs'].to(device='cuda')
            
            output_val, mean_activity, mean_activity_conv1, output_gates_val_conv1, mean_activity_egru_relu, box_hid_mean, cls_hid_mean = inference_step(data,net,ssd_head,box_coder)
            print('inference end')
            indices = (output_gates_val_conv1[0]).nonzero().cpu()
            x_indices = indices[:, 2]
            y_indices = indices[:, 3]
            
                        
            x_starts = torch.clamp(2 * x_indices - 1, min=0, max=receptive_field_input[0].size(0) )
            x_ends = torch.clamp(2 * x_indices + 1, min=0, max=receptive_field_input[0].size(0))
            y_starts = torch.clamp(2 * y_indices - 1, min=0, max=receptive_field_input[0].size(1) )
            y_ends = torch.clamp(2 * y_indices + 1, min=0, max=receptive_field_input[0].size(1))
            
            
            x_starts = np.array(x_starts)
            x_ends = np.array(x_ends)
            y_starts = np.array(y_starts)
            y_ends = np.array(y_ends)

            receptive_field_input[0].cpu().numpy()
            increments = np.ones_like(receptive_field_input[0])

            receptive_field_input[0][x_starts[:, None], y_starts[:, None]] += increments[x_starts[:, None], y_starts[:, None]]
            receptive_field_input[0][x_starts[:, None], y_ends[:, None]] += increments[x_starts[:, None], y_ends[:, None]]
            receptive_field_input[0][x_ends[:, None], y_starts[:, None]] += increments[x_ends[:, None], y_starts[:, None]]
            receptive_field_input[0][x_ends[:, None], y_ends[:, None]] += increments[x_ends[:, None], y_ends[:, None]]

            fig, ax = plt.subplots(figsize=(10, 6))
            image = ax.imshow(np.uint8(receptive_field_input[0]), cmap='gray')
            print(receptive_field_input[0])
            plt.colorbar(image, ax=ax)
            plt.show()
            
            pbar.update(1)
    
